<a href="https://colab.research.google.com/github/Eddy109109/Dacon_Competition/blob/master/20210310_%EA%B5%AD%ED%9A%8C%EC%9E%85%EB%B2%951.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 27.1MB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5 

In [4]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText
import fasttext.util

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 27.61 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
     |████████████████████████████████| 71kB 5.2MB/s 
  Created wheel for fastText: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3098549 sha256=f640c811daf85eb94967edd257baa4c80d6957e10ee437ca9b11e0ef2656e506
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fastText


In [6]:
!pip install selenium

     |████████████████████████████████| 911kB 8.5MB/s 


In [7]:
import re
import json
import time
import konlpy
import gensim 
import glob, os
import requests
import fasttext
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
import urllib.request
import matplotlib as mpl
from matplotlib import rc
from konlpy.tag import Mecab
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.font_manager as fm
from matplotlib import font_manager, rc
from wordcloud import ImageColorGenerator
from IPython.display import Image, display
from sklearn.metrics import silhouette_score
from selenium.webdriver.common.keys import Keys
from gensim.models import Word2Vec, FastText, Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud, STOPWORDS as stopwords
from selenium.webdriver.common.action_chains import ActionChains
%matplotlib inline
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

#함수모음


In [8]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

In [9]:
def out_stopwords(data, list_stopwords):
    data = data.split(" ")
    list_stopwords = list_stopwords
    data = [token for token in data if token not in list_stopwords]
    return data

In [10]:
def show_bills(bills_id, df_bills):
    for bill_id in bills_id:
        print(df_bills[df_bills['bill_id']==int(bill_id)]['법안명_법안내용'].iloc[0])
        print("*"*80)        
        
def input2vec(user_input, stopword_list, word2vec_word_dict):
    tokenized_input = tokenized_mecab(user_input)
    tokenized_input_outStopwords = out_stopwords(tokenized_input, stopword_list)
    
    list_vector = []
    for word in tokenized_input_outStopwords:
        if word in word_dict.keys():
            list_vector.append(word_dict[word])
    user_vector = (np.sum(list_vector, axis=0) / len(list_vector)).tolist()
    return user_vector

def similar_senators(user_vector, df_snt):
    similarity = {}
    for idx in df_snt.index:
        if df_snt.loc[idx]['vector'] != '0.0':
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array([float(i) for i in df_snt.loc[idx]['vector'][1:-1].split(', ')]).reshape(1,-1))
        similarity[str(df_snt.loc[idx]['id'])] = float(sim)
    similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
    rating = [key for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
    top_5 = rating[:5]
    result = {}
    for i in top_5:
        result[i] = str(abs(round((similarity[i]*100), 2))) + "%"
        
    return result

def similar_bills(user_vector, df_bills, df_snt, fasttext_word_dict, snt_ids):
    senators = {}
    for id_ in tqdm(snt_ids):
        id_ = int(id_)
        NAME_HAN = df_snt[df_snt['id']==id_]['NAME_HAN'].iloc[0]
        bills_snt_ids = df_bills[df_bills['senators_list'].str.contains(NAME_HAN)]['BILL_ID'].values
        
        similarity = {}
        for bill_id in bills_snt_ids:
            sim = cosine_similarity(np.array(user_vector).reshape(1,-1), np.array([float(i) for i in df_bills[df_bills['BILL_ID']==bill_id].iloc[0]['vector'][1:-1].split(', ')]).reshape(1,-1))
            similarity[df_bills[df_bills['BILL_ID']==bill_id]['BILL_ID'].iloc[0]] = float(sim)
        similarity = {key: value for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)}
        rating = [str(key) for key, value in sorted(similarity.items(), key=lambda item: item[1], reverse=True)]
        top_5 = rating[:3]
        result = {}
        for i in top_5:
            result[i] = str(abs(round((similarity[i]*100), 2))) + "%"
        senators[id_] = result
    
    return senators

def show_bills(senators_id_bills, df_bills, df_snt):
    result = []
    
    for snt_id in senators_id_bills:
        name = df_snt[df_snt['id'] == snt_id]['NAME'].iloc[0]
        for bill_id in list(senators_id_bills[snt_id]):
            dict_temp = {}
            dict_temp['발의자'] = name
            dict_temp['법안명'] = df_bills[df_bills['BILL_ID']==bill_id]['BILL_NAME'].iloc[0]
            dict_temp['법안내용'] = df_bills[df_bills['BILL_ID']==bill_id]['bill_content'].iloc[0]
            result.append(dict_temp)
    return result

def show_results(df_results, dict_images):
    for snt in df_results['발의자'].unique():
        display(Image('./images/'+snt+'.jpg'))
        print("{} 의원".format(snt))
        df_temp = df_results[df_results['발의자']==snt]
        for idx in df_temp.index:
            print("*"*30, df_temp.loc[idx]['법안명'], "*"*30)
            print(df_temp.loc[idx]['법안내용'], '\n')

# 정보 크롤링

In [16]:
dict_data = {}
for i in glob('/content/data/*.csv'):
    try:
        dict_data[i.split('/')[3].split('.csv')[0]] = pd.read_csv(i)
    except:
        dict_data[i.split('/')[3].split('.csv')[0]] = pd.read_csv(i, encoding='cp949')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
dict_data

{'new_people':       DAESU                                                DAE DAE_NM NAME  \
 0        21  제19대국회의원(경남 창원시성산구) 정당_66\r\n제21대국회의원(경남 창원시성산...  21대의원  강기윤   
 1        21                              제21대국회의원(대구동구을) 정당_74  21대의원  강대식   
 2        21                            제21대국회의원(경기안양시만안구) 정당_7  21대의원  강득구   
 3        21                             제21대국회의원(경남진주시을) 정당_74  21대의원  강민국   
 4        21                               제21대국회의원(비례대표) 정당_30  21대의원  강민정   
 ...     ...                                                ...    ...  ...   
 5600      1                                 제헌국회의원(김제을)조선민족청년단   제헌의원  홍희종   
 5601      1                               제헌국회의원(순천갑)대한촉성농민총연맹   제헌의원  황두연   
 5602      1                 제헌국회의원(여수을)무소속\r\n제2대국회의원(여천)대한국민당   제헌의원  황병규   
 5603      1                                      제헌국회의원(진양)무소속   제헌의원  황윤호   
 5604      1  제헌국회의원(평창)대한독립촉성국민회\r\n제4대국회의원(평창)무소속\r\n제6대국회...   제헌의원  황호현   
 
      NAME_HAN   JA   HO          BI

##-1 사진


In [12]:
def download(URL, short_name):
    list_temp = []
    short_name = short_name[0].text.split(' ',1)[-1]
    if short_name not in list_temp:
        list_temp.append(short_name)
        fullName = "images/" + short_name + ".jpg"
        urllib.request.urlretrieve(URL, fullName)   
    else:
        fullName = "images/" + short_name + " (2).jpg"
        urllib.request.urlretrieve(URL, fullName)   

In [14]:
driver = webdriver.Chrome('chromedriver',options=options)

NameError: ignored

In [13]:
# 열러라 국회 의원 접속
driver.get('http://watch.peoplepower21.org/?act=&mid=AssemblyMembers&vid=&mode=search&name=&party=&region=&sangim=&gender=&age=&elect_num=')

NameError: ignored

In [ ]:
start_time = time.time()

for page in range(1,11):
    if page != 1:
        driver.get('http://watch.peoplepower21.org/?mid=AssemblyMembers&mode=search&party=&region=&sangim=&gender=&elect_num=&page=' + str(page))
    for person in range(2,32):
        short_name = driver.find_elements_by_css_selector('#content > div.col-md-8 > div:nth-child('+ str(person)+')> div > a:nth-child(2) > h4')
        image = driver.find_elements_by_tag_name('#content > div.col-md-8 > div:nth-child(' + str(person)+ ') > div > a:nth-child(1) > img')
        src = image[0].get_attribute('src')
        download(src, short_name)
print("---{}초 걸렸슴둥---".format(time.time()-start_time))

##-2 법안내용

In [ ]:
sug = dict_data['suggest']

In [ ]:
dict_crawl_data = {}
for idx in tqdm(sug.index):
    bill_id = sug.loc[idx]['BILL_ID']
    url = sug.loc[idx]['DETAIL_LINK']

    res = requests.get(url)

    soup = BeautifulSoup(res.text, 'html.parser')
    container = soup.select('#summaryContentDiv')
    if len(container) == 0:
        dict_crawl_data[bill_id] = '내용없음'
    else:
        contents = container[0].text.strip()
        dict_crawl_data[bill_id] = contents

In [ ]:
sug['bill_content'] = sug['BILL_ID'].map(dict_crawl_data)

In [ ]:
sug.to_csv('data/suggest ver4.0 (members_added')